In [2]:
%matplotlib inline
import os
import sys
import itertools
import re
import pprint
import imp

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.misc as climamisc
imp.reload(climamisc)

<module 'climatools.misc' from '/nuwa_cluster/home/jackyu/climatools/climatools/misc.py'>

In [3]:
def fortran_files():
    dir_cam = '/nuwa_cluster/home/jackyu/climate_models/cesm1_2_2/models/atm/cam/src/physics/cam/'
    cam_files = ['aer_rad_props.F90', 'rad_constituents.F90', 'phys_prop.F90']
    fpaths = [os.path.join(dir_cam, file) for file in cam_files]
    return fpaths

In [4]:
pprint.pprint(climamisc.Fortran_subroutine_relations_from_files(paths_fortran = fortran_files()))

{'aer_optics_log': {'childs': [], 'parents': []},
 'aer_optics_log_rh': {'childs': [], 'parents': ['bulk_props_init']},
 'aer_rad_props_init': {'childs': ['rad_cnst_get_info',
                                   'addfld',
                                   'add_default',
                                   'phys_getopts'],
                        'parents': []},
 'aer_rad_props_lw': {'childs': ['modal_aero_lw',
                                 'qsat',
                                 'rad_cnst_get_info',
                                 'pbuf_get_field',
                                 'rad_cnst_get_aer_mmr',
                                 'rad_cnst_get_aer_props',
                                 'endrun'],
                      'parents': []},
 'aer_rad_props_sw': {'childs': ['get_volcanic_rad_props',
                                 'get_hygro_rad_props',
                                 'qsat',
                                 'get_nonhygro_rad_props',
                            